In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
ALL_PRICES = pd.read_csv("2020-07-26-sp500-prices.csv")
ALL_PRICES.index = pd.to_datetime(ALL_PRICES.iloc[:,0])     # set index to symbol
ALL_PRICES.drop(columns=["Unnamed: 0"])                     # remove initial index column
ALL_PRICES = ALL_PRICES.select_dtypes(include=["float64"])  # filter NA

PROFILES = pd.read_csv("2020-07-26-profiles.csv")
SP500_PROFILES = PROFILES[PROFILES.symbol.isin(set(ALL_PRICES.columns))]

In [3]:
def get_diffs(start, end):
    df = ALL_PRICES[(ALL_PRICES.index >= start) & (ALL_PRICES.index < end)] # filter
    diffs = np.log(df)                                                      # log prices
    diffs = -diffs.diff()                                                   # log diffs
    return diffs

In [4]:
SP500_PROFILES.groupby(by="sector").count().symbol

sector
Basic Materials           22
Communication Services    25
Consumer Cyclical         65
Consumer Defensive        35
Energy                    25
Financial Services        69
Healthcare                62
Industrials               73
Real Estate               31
Technology                67
Utilities                 28
Name: symbol, dtype: int64

In [5]:
def get_prices_by_sector(start, end):
    diffs = get_diffs(start, end)
    prices_by_sector = {}
    for sector in SP500_PROFILES.sector.unique():
        symbols = SP500_PROFILES.symbol[SP500_PROFILES.sector == sector]
        prices_by_sector[sector] = diffs[symbols]
    return prices_by_sector

def get_medians_by_sector(start, end):
    diffs = get_diffs(start, end)
    for sector in SP500_PROFILES.sector.unique():
        symbols = SP500_PROFILES.symbol[SP500_PROFILES.sector == sector]
        median = diffs[symbols].median(axis=1)
        medians_by_sector[sector] = median
    return medians_by_sector

In [6]:
def get_performance_by_sector(year):
    start = f"{year}-01-01"
    end = f"{year}-12-31"
    prices_by_sector = get_prices_by_sector(start, end)
    performance = {}
    for sector, returns in prices_by_sector.items():
        performance[sector] = [returns.sum().median()]
    df = pd.DataFrame.from_dict(performance).transpose()
    df.columns = [year]
    return df

In [46]:
def get_performance_range_by_sector(year):
    start = f"{year}-01-01"
    end = f"{year}-12-31"
    prices_by_sector = get_prices_by_sector(start, end)
    performance = {}
    for sector, returns in prices_by_sector.items():
        perc_20 = np.percentile(returns.sum(), 20).round(4)
        perc_50 = np.percentile(returns.sum(), 50).round(4)
        perc_80 = np.percentile(returns.sum(), 80).round(4)
        performance[sector] = [perc_20, perc_50, perc_80]
    df = pd.DataFrame.from_dict(performance).transpose()
    df.columns = [f"{year} lo",f"{year} med", f"{year} hi"]
    return df

In [28]:
performance = None
for year in range(2010, 2020):
    year_performance = get_performance_by_sector(year)
    performance = year_performance if performance is None else performance.join(year_performance)

In [47]:
performance_range = None
for year in range(2010, 2020):
    year_performance_range = get_performance_range_by_sector(year)
    performance_range = year_performance_range if performance_range is None else performance_range.join(year_performance_range)

In [44]:
performance_range.to_csv("")

,2010 lo,2010 hi,2011 lo,2011 hi,2012 lo,2012 hi,2013 lo,2013 hi,2014 lo,2014 hi,2015 lo,2015 hi,2016 lo,2016 hi,2017 lo,2017 hi,2018 lo,2018 hi,2019 lo,2019 hi
Industrials,0.0119,0.3766,-0.1874,0.1294,0.0060,0.2571,0.1520,0.4256,0.0000,0.2594,-0.2477,0.1576,0.0947,0.3036,0.0248,0.3305,-0.2663,0.0110,0.1088,0.3975
Consumer Cyclical,0.0085,0.4670,-0.1617,0.2635,0.0000,0.3942,0.1052,0.4490,0.0000,0.2959,-0.2048,0.2112,-0.1289,0.2056,-0.0085,0.3515,-0.4059,0.1186,0.0949,0.3465
Technology,-0.0249,0.3988,-0.2645,0.0902,-0.0257,0.2386,0.0260,0.3976,0.0230,0.3186,-0.1742,0.2084,0.0412,0.2862,0.1465,0.3896,-0.1717,0.1687,0.1499,0.5229
Healthcare,-0.0369,0.2671,-0.0970,0.1972,0.0205,0.2601,0.1303,0.4887,0.1383,0.3319,-0.0396,0.1744,-0.1916,0.1756,0.0570,0.3693,-0.1869,0.1644,0.1109,0.3601
Consumer Defensive,0.0000,0.2693,-0.0012,0.2385,0.0000,0.1336,0.0857,0.3443,0.0593,0.2110,0.0086,0.2140,-0.0265,0.1583,-0.0828,0.2326,-0.3925,0.0396,0.0806,0.3683
Utilities,-0.0958,0.1359,0.0509,0.1996,-0.0613,0.0901,0.0083,0.1926,0.2154,0.2947,-0.1832,0.0114,0.0766,0.1992,0.0158,0.1536,-0.0011,0.0938,0.1117,0.3201
Financial Services,-0.0075,0.2560,-0.3628,0.0645,0.0285,0.2311,0.2224,0.4543,0.0297,0.1784,-0.0991,0.0971,0.1131,0.2949,0.0690,0.2889,-0.3021,0.0235,0.1494,0.3463
Real Estate,0.0925,0.3306,-0.1056,0.1815,0.0220,0.2127,-0.1022,0.0753,0.2334,0.3419,-0.0657,0.1454,-0.0239,0.1883,-0.0614,0.1663,-0.1734,0.0338,0.1037,0.3539
Basic Materials,0.0293,0.3404,-0.1812,0.0508,-0.0323,0.2673,0.0065,0.2640,-0.0598,0.1506,-0.2864,0.1139,0.0000,0.4066,0.0059,0.2959,-0.3060,-0.0036,-0.0476,0.3624
Communication Services,0.0000,0.2920,-0.0494,0.0922,-0.0046,0.2202,0.1212,0.4952,-0.1364,0.2368,-0.1726,0.3779,-0.0082,0.2141,-0.1540,0.3849,-0.3113,0.0345,0.1025,0.2802


In [48]:
performance_range.to_csv("performance_range.csv")